In [1]:
%matplotlib inline
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc
import gc  # 导入垃圾回收模块
from tqdm import tqdm
sc.settings.verbosity = 3
sc.logging.print_header()
sc.set_figure_params(dpi=100, dpi_save=600)

/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/umap/d

scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.24.4 scipy==1.11.1 pandas==1.3.5 scikit-learn==1.3.0 statsmodels==0.14.0 python-igraph==0.10.6 louvain==0.8.0 pynndescent==0.5.10


In [2]:
#无子文件夹运行这个

base_dir = '/mnt/f/ZJH_KRAS/'
input_dir = os.path.join(base_dir, '{}')
output_dir = os.path.join(base_dir, '{}/')
# 获取所有以 "GSM" 开头的文件夹
folders = [f for f in os.listdir(base_dir)]
for folder in tqdm(folders, desc="Processing folders..."):
    try:
        input_path = input_dir.format(folder)
        output_path = output_dir.format(folder)
        os.makedirs(output_path, exist_ok=True)

        # 检查Scrublet结果是否已经存在
        if os.path.exists(os.path.join(output_path, 'raw_after_Scrublet.h5ad')):
            print(f"{output_path} 已经存在，跳过...")
            continue
        # Check if the Scrublet results already exist
        # Load counts matrix and gene list
        counts_matrix = scipy.io.mmread(os.path.join(input_path, 'matrix.mtx')).T.tocsc()
            # 识别features.tsv或genes.tsv文件
        features_file = 'features.tsv' if os.path.exists(os.path.join(input_path, 'features.tsv')) else 'genes.tsv'
        genes = np.array(scr.load_genes(os.path.join(input_path, features_file), delimiter='\t', column=1))
        print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
        print('Number of genes in gene list: {}'.format(len(genes)))

        # Initialize Scrublet object
        scrub = scr.Scrublet(counts_matrix=counts_matrix, sim_doublet_ratio=2.0, n_neighbors=None, expected_doublet_rate=0.06)

        # Run the default pipeline
        doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, min_cells=3, min_gene_variability_pctl=85, n_prin_comps=30)

        # Get 2-D embedding to visualize the results
        scrub.set_embedding('UMAP', scr.get_umap(X=scrub.manifold_obs_, n_neighbors=10, min_dist=0.3))

        # Export Scrublet results
        barcodes = pd.read_csv(os.path.join(input_path, 'barcodes.tsv'), delimiter='\t', header=None)
        result = pd.DataFrame([barcodes.iloc[:,0], doublet_scores, predicted_doublets], index=["barcode", "doublet_scores","predicted_doublets"]).T
        result.to_csv(output_path + 'Scrublet_results.csv')

        # Save as h5ad after Scrublet
        adata = sc.read_mtx(os.path.join(input_path, 'matrix.mtx')).T
        adata.var_names = [line.strip() for line in open(os.path.join(input_path, features_file))]
        adata.obs_names = [line.strip() for line in open(os.path.join(input_path, 'barcodes.tsv'))]
        adata.var_names_make_unique()
        Cells=pd.read_csv(output_path + 'Scrublet_results.csv', index_col='barcode')
        adata.obs['predicted_doublets']=Cells['predicted_doublets']
        SAS=adata[adata.obs['predicted_doublets']==False]
        SAS.write(output_path + 'raw_after_Scrublet.h5ad', compression='gzip')
        # Clear memory
        del counts_matrix, genes, scrub, doublet_scores, predicted_doublets, barcodes, result, adata, Cells
        plt.close('all')
        gc.collect()
    except FileNotFoundError:
        print(f"File not found for {folder}. Skipping...")
        continue


Processing folders...:   0%|                                                                                                                        | 0/10 [00:00<?, ?it/s]

Counts matrix shape: 1472 rows, 30893 columns
Number of genes in gene list: 30893
Preprocessing...


/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/scrublet/helper_functions.py:239: RuntimeWarning: invalid value encountered in log
  gLog = lambda input: np.log(input[1] * np.exp(-input[0]) + input[2])
/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/scrublet/helper_functions.py:252: RuntimeWarning: invalid value encountered in sqrt
  CV_input = np.sqrt(b);


Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.41
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 37.4%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.5%
Elapsed time: 2.2 seconds



Processing folders...:  10%|███████████▏                                                                                                    | 1/10 [00:30<04:30, 30.05s/it]

...


Processing folders...:  90%|████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 9/10 [08:54<00:46, 46.99s/it]

Counts matrix shape: 2352 rows, 30893 columns
Number of genes in gene list: 30893
Preprocessing...


/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/scrublet/helper_functions.py:239: RuntimeWarning: invalid value encountered in log
  gLog = lambda input: np.log(input[1] * np.exp(-input[0]) + input[2])
/home/yqcj/miniconda3/envs/scrublet/lib/python3.9/site-packages/scrublet/helper_functions.py:252: RuntimeWarning: invalid value encountered in sqrt
  CV_input = np.sqrt(b);


Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.47
Detected doublet rate = 0.6%
Estimated detectable doublet fraction = 22.7%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 2.4%
Elapsed time: 3.3 seconds


Processing folders...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [09:32<00:00, 57.21s/it]
